In [1]:
from pymilvus import connections, Collection

collection_name = "example"
connections.connect(
    host="127.0.0.1",
    port="19530"
)

collection = Collection(name=collection_name)

collection.load()



In [2]:
from sentence_transformers  import SentenceTransformer
import os

pwd = %pwd
transformerPath = os.path.join(pwd, "Qwen3-Embedding-0.6B")
transformer = SentenceTransformer(transformerPath)

In [3]:
from IPython.display import display
import pandas as pd
from textwrap import wrap

In [4]:
result = collection.query(
    expr="upvotes > 1000",
    limit=5,
    output_fields=["upvotes", "question", "answer"]
)

display(pd.DataFrame(result))


,upvotes,question,answer,id
0,2292,你对青年警察有什么建议呢？,看有这么多慌张的弟弟妹妹。再多数几句。 1.遴选。我所在单位之前遴选到市局是不需要工作年限的...,458650682755122888
1,3896,如果你魂穿在了戚夫人身上，时间点是刘邦死后，该如何自保，不被做成人彘？,很简单。其实戚夫人在刘邦死后要保命本来就是简单难度。 首先吕雉本来是一个贤德的大妇，对于刘邦...,458650682755122889
2,3971,为什么老一辈会认为健身出来是畸形身材？,正常的健身： 打拳的 打球的 游泳的 跑步的 踢球的 畸形的健身： 老一辈觉得怎么样我不...,458650682755122890
3,1187,为什么 苹果Mac 电脑不粘贴 Intel 标志?,谢邀。尽管现在好几家强势笔记本厂商没有贴Intel标志，但苹果Mac确实是第一个这么做的。这...,458650682755122891
4,4725,为什么有些胖子明明瘦下来会很好看，却仍然在肥胖的道路上越走越远？,曾经，我就是一个大胖子 评论里120多斤的算啥胖啊，在我看也就微胖略丰满吧 爆个照（最胖时期...,458650682755122892


In [5]:
def searchToPd(res):
    data = []
    for hits in res:
        for hit in hits:
            data.append({
                "id": hit.entity.get("id"),
                "upvotes": hit.entity.get("upvotes"),
                "question": hit.entity.get("question"),
                "answer": hit.entity.get("answer"),
                "distance": hit.distance
            })

    return pd.DataFrame(data)

In [6]:
import time
q = "three-body"

st = time.perf_counter()
q_embedding = transformer.encode(q)
display(f"{q} embedding took: {time.perf_counter() - st} seconds")

res = collection.search(
    data=[q_embedding],
    limit=5,
    param={"metric_type": "L2", "params": {"nprobe": 3}},
    anns_field="question_embedding",
    output_fields=["upvotes", "question", "answer"],
)

searchToPd(res)

'three-body embedding took: 4.444132834032644 seconds'

,id,upvotes,question,answer,distance
0,458650682755123650,6709,地球广播三体坐标后，三体人为什么不报复？,你是一个成年壮汉，孤身一人在黑暗的无边原始森林里行走，森林里到处潜伏着猛兽，你不敢出声，经过...,1.029931
1,458650682755123855,19000,我们生活在三维空间，是偶然还是必然？,好久没有看到这么有趣的问题了！ 首先明确一个概念，我们是生活在3+1维度的宇宙中——3维的空...,1.049264
2,458650682755123660,2003,《三体》三部中最大的 bug 是什么？,红岸基地位于东北，根本看不到南门二，三体的回复叶文洁不可能收到，故事从一开始就不成立。 不过...,1.057100
3,458650682755123709,2109,你觉得哪个韩国组合关系不怎样？,不小心1.3k赞了，想我家哥哥，快点回来开演唱会吧，我一定会去看的噢！我已经很久没有关注过c...,1.102375
4,458650682755123039,7795,为何电子不会坠入原子核？,要是放在一百多年前，题主提出了这个问题，那么题主就可能是量子星空的一颗星星了，可以和波尔薛定...,1.118779


In [7]:
q = "太阳系遭二向箔降维打击"
q_embedding = transformer.encode(q)
res = collection.search(
    data=[q_embedding],
    limit=10,
    param={"metric_type": "L2", "params": {"nprobe": 3}},
    anns_field="answer_embedding",
    output_fields=["upvotes", "question", "answer"],
)

df = searchToPd(res)
display(df)
display(wrap(df.loc[0, "answer"]))

,id,upvotes,question,answer,distance
0,458650682755123580,6501,《三体》到底有多惊艳？,不要总是关注什么二向箔，光粒这些的。 三体真正牛逼的预言是这几个： 大号流感刺杀重要人物。 ...,0.838995
1,458650682755123655,3139,刘慈欣可否算一己之力带歪外星人友好论变为外星人威胁论？,俺寻思着好莱坞大片里，外星人跑地球这旮瘩来，有几回不是要毁灭人类的………,0.946025
2,458650682755122973,1460,为什么战锤中四小贩能拉这么多人堕落？任何一个理智人都应该知道它们不是什么好东西吧？,我的建议是不要把大家开玩笑时候说的“四小贩”当真了，搞得好像四小贩捞人真像商人一样跟你讨价还...,0.976969
3,458650682755123927,25000,能够随意改变物体颜色这一能力是不是很鸡肋？,鸡肋？ 你将地表物质的颜色都改成白色，那用不了一年，全人类都会跪在你面前叫爸爸，求你结束地球...,0.984130
4,458650682755123028,2252,如何评价阿里员工发文《致阿里》：我想偷懒驴之常情，你不想我偷懒人之常情，但不能一边抽我一边让...,,0.999999
5,458650682755123578,1337,国家有没有处理灵异事件的特殊部门？,,0.999999
6,458650682755123763,39000,张静初陶虹舒淇倡导吃素遭网友嘲讽，对此事你怎么看？你认为吃素食真的对身体有好处吗？,,0.999999
7,458650682755123778,20000,为什么总叫央视六套六公主？,,0.999999
8,458650682755123599,4573,如何评价「快手」APP？,跟刚下载知乎一样，让我感觉看到了更大的世界,1.000973
9,458650682755122893,1304,地球资源使用已经超过年预估值了，为什么人类依旧我行我素?,因为资源不足本来就是一个“伪”属性大于80%的命题。 几百年前的古人在用什么资源，现在你在用...,1.003457


['不要总是关注什么二向箔，光粒这些的。 三体真正牛逼的预言是这几个： 大号流感刺杀重要人物。 环保组织阻止科技发展。 圣母文化摧毁抵抗意志。',
 '再补充两个评论里面的： 无处不在的精准广告推送和共享单车。。。。 我感觉第一批三体读者都错失了一个亿。。。。']